# ALB Checker

The purpose of this notebook is to showcase whether we can dissect which ALBs will be interested in a particular WIP action.

I will then try and extract which aspects of the action that ALB will be interested in
Then try and align that aspect with a section that ALB cares about.

In [1]:
from langchain_openai import ChatOpenAI
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema.messages import SystemMessage
import yaml
import os
import json
import pandas as pd
import autogen
import yaml
import os

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    # filter_dict={
    #     "model": ["gpt-3.5-turbo"],
    },
)

gpt4_config = {
    "seed": 1337,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
}

In [5]:
user_proxy = autogen.UserProxyAgent(
    name="admin",
    system_message=f"""You coordinate the tasks.

    You take a user input which is a plain text document written in prose.
    And run it through the below workflow to each member from organisations, with the aim of identifying which aspects of the document will be of interest to that member.

    Workflow:
    - Identify whether the document applies to the organisation being assessed. It is perfectly acceptable if it is not.
    - Identify which specific sections of the document the organisation is interested in. Quote the section of the document.
    - For each quoted section, explain why the organisation will be interested in the document

    The current list of organisations is: ['Natural England', 'Forestry Commission', 'Animal and Plant Health Agency', 'Centre for Environment, Fisheries and Aquaculture Science', 'Environment Agency', 'Marine Management Organisation']

    One you have all the feedback from each expert and thier organisation summarise any repeat sections between multiple organisations - but highlight which organisations are impacted and why.
    And return all areas the organisation thought needed rework. We are particularly interested in overlaps between agencies so please highlight those.

    Finally - output a list of the organisation which have highlighted they are impacted.

    Once you have collected feedback from everyobe - submit "TERMINATE" into the chain.
    """,
    human_input_mode = "NEVER",
    llm_config=gpt4_config,
    code_execution_config=False,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE")
)

organisation_template = '''You are the specialist from an organisation.
    You read the user submitted document and output whether based on the description of your organisation you are impacted by the document.
    It is perfectly fine to say your are not impacted, but we are looking for broad views so if there is anything related please mention it.

    If you are impacted, quote which sections of the document you are interested in.
    - Explain why you are interested in that section.
    - Output each specific section as a numbered point, with the explaination being text under that point. Make separate explainations for each section.

    Below is a description of your organisation (from a markdown file):'''


organisation_natural_england = autogen.AssistantAgent(
    name="organisation_natural_england",
    llm_config=gpt4_config,
    system_message=f'''{organisation_template}
# Natural England
## Role:
This body is focused on conserving and enhancing the natural environment, including biodiversity and landscapes.
## Current Aims:
Natural England aims to improve habitat conservation, increase access to nature, and promote environmental stewardship.
## Environmental Impact:
By protecting species and habitats, Natural England contributes to maintaining biodiversity and enhancing ecosystem services.
## Expectations from Farmers and Landowners:
Natural England promotes the adoption of agri-environment schemes that support wildlife-friendly farming and habitat restoration.
''',
)

organisation_forestry_commission = autogen.AssistantAgent(
    name="organisation_forestry_commission",
    llm_config=gpt4_config,
    system_message=f'''{organisation_template}
    Below is a description of your organisation (from a markdown file):
## Forestry Commission
# Role:
The Forestry Commission is responsible for protecting, expanding, and promoting the sustainable management of woodlands and forests.
# Current Aims:
Increasing woodland cover, promoting sustainable forestry practices, and enhancing the role of forests in combating climate change are key priorities.
## Environmental Impact:
The Commission's work supports carbon sequestration, biodiversity conservation, and flood mitigation through sustainable forest management.
## Expectations from Farmers and Landowners:
Farmers and landowners are encouraged to plant trees, manage woodlands sustainably, and participate in afforestation initiatives.
''',
)

organisation_animal_and_plant_health_agency  = autogen.AssistantAgent(
    name="organisation_animal_and_plant_health_agency",
    llm_config=gpt4_config,
    system_message=f'''{organisation_template}
# Animal and Plant Health Agency (APHA)
## Role:
APHA safeguards animal and plant health, ensuring food safety and biosecurity in England.
## Current Aims:
Key objectives include preventing the spread of animal diseases, regulating veterinary medicines, and protecting crops from pests and diseases.
## Environmental Impact:
APHA's work contributes to the sustainable management of agricultural resources and protects ecosystems from invasive species.
## Expectations from Farmers and Landowners:
APHA expects compliance with biosecurity measures, responsible use of veterinary products, and participation in disease control programs.
''',
)

organisation_environment_fisheries_aquaculture  = autogen.AssistantAgent(
    name="organisation_environment_fisheries_aquaculture",
    llm_config=gpt4_config,
    system_message=f'''{organisation_template}
# Centre for Environment, Fisheries and Aquaculture Science (CEFAS)
## Role:
CEFAS provides scientific advice on marine and freshwater environments.
## Current Aims:
The agency focuses on supporting sustainable fisheries, understanding climate change impacts, and protecting aquatic environments.
## Environmental Impact:
CEFAS's research underpins policies that aim to protect marine biodiversity and support sustainable aquaculture.
## Expectations from Farmers and Landowners:
CEFAS works with stakeholders in the aquaculture industry to promote practices that minimize environmental impacts.
''',
)


marine_management_organisation  = autogen.AssistantAgent(
    name="marine_management_organisation",
    llm_config=gpt4_config,
    system_message=f'''{organisation_template}
# Marine Management Organisation (MMO)
## Role:
MMO manages England's marine resources, balancing conservation and economic activities.
## Current Aims:
The MMO focuses on sustainable fisheries, protecting marine habitats, and regulating marine development.
## Environmental Impact:
By ensuring sustainable use of marine resources, the MMO helps maintain healthy marine ecosystems.
## Expectations from Farmers and Landowners:
While more relevant to coastal landowners, the MMO encourages responsible land use that doesn't negatively impact marine environments, such as preventing runoff that could contribute to marine pollution.
''',
)

organisation_environment_agency  = autogen.AssistantAgent(
    name="organisation_environment_agency",
    llm_config=gpt4_config,
    system_message=f'''{organisation_template}
# Environment Agency (EA)
## Role:
The EA is responsible for protecting and improving the environment in England. It oversees flood defenses, manages water resources, regulates pollution, and monitors environmental quality.
## Current Aims:
Its current goals include tackling climate change impacts, enhancing biodiversity, and improving water quality.
## Environmental Impact:
The EA plays a significant role in mitigating environmental risks, reducing pollution, and ensuring sustainable use of natural resources.
## Expectations from Farmers and Landowners:
The EA encourages sustainable farming practices, efficient water usage, and compliance with regulations to reduce pollution and protect natural habitats.
''',
)


In [6]:
groupchat = autogen.GroupChat(agents=[user_proxy,
                                      organisation_natural_england,
                                      organisation_forestry_commission,
                                      organisation_animal_and_plant_health_agency,
                                      organisation_environment_fisheries_aquaculture,
                                      organisation_environment_agency,
                                      marine_management_organisation],
                              messages=[],
                              admin_name = user_proxy,
                              max_round=10)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)


policy_content = '''This is an action in the Sustainable Farming Incentive (SFI) scheme: expanded offer for 2024. You must read the SFI scheme information to understand the scheme rules and how to apply. 3 years £6 per hectare (ha) per year and £97 per SFI agreement per year This action’s aim is that you: You can do this action on agricultural land located below the moorland line that’s: For land above the moorland line, you can do CMOR1 to complete a survey to identify, assess and record the soil, vegetation and historic and archaeological features. Total available area in a land parcel. This action is static. This means you must do it at the same location each year of this action’s duration. You must: You can use the following to get paid for this action: To test the SOM for small land parcels, you can combine them into a single sample area if the soil type and past management is similar. This means you can take soil from each land parcel and combine it into a single sample to test the SOM. If you need to take soil samples on areas within a land parcel containing historic or archaeological features, including scheduled monuments, you must follow the requirements explained in section 5.6: ‘Land with historic or archaeological features’ of the SFI scheme guidance. To help improve national data on the condition of England’s soils, Defra or the RPA may ask you to provide some information about your soil assessment and SOM testing results during this action’s 3-year duration. This data is one of the public goods that this action is paying for and part of wider government measures in relation to our soils policy and strategy. We’ll publish guidance on GOV.UK about what information is needed and how to supply it. For the soil assessment and soil management plan, you must: For the SOM testing, each year of this action’s 3-year duration you must: If the timing meets the requirements above, you can do the SOM testing at the same time as your other routine soil testing required in the farming rules for water. If you enter land into this action which has not been tested for SOM within the last 5 years, you should try to do the SOM testing on that land during the first 12 months of your SFI agreement. If the laboratory that tests your soil samples does not have capacity to do the tests during the first 12 months of this action’s duration, you must: It’s up to you how you do this action, as long as you: Advice to help you do this action will be published before applications are fully launched from summer 2024. It will not be part of this action’s requirements. You must keep: You must supply this evidence when we ask for it. You can do the following actions or options on the same area in a land parcel as this action. Some actions or options can only be done on the same area if they’re done at a different time of year to this action. For example, winter cover followed by a summer companion crop. Read ‘What to do’ and ‘When to do it’ to find out when this action must be done. If an action or option cannot be located on the same area, you may be able to do it on a different area in the same land parcel. Read section 6. ‘Eligible land in other funding schemes’ for more information. You can do the following actions or options on the eligible boundaries of a land parcel entered into this action: Don’t include personal or financial information like your National Insurance number or credit card details.'''
user_proxy.initiate_chat(
    manager,
    message=f"""run the worklow and output feedback for the following document: {policy_content}'''
    """,)

admin (to chat_manager):

run the worklow and output feedback for the following document: This is an action in the Sustainable Farming Incentive (SFI) scheme: expanded offer for 2024. You must read the SFI scheme information to understand the scheme rules and how to apply. 3 years £6 per hectare (ha) per year and £97 per SFI agreement per year This action’s aim is that you: You can do this action on agricultural land located below the moorland line that’s: For land above the moorland line, you can do CMOR1 to complete a survey to identify, assess and record the soil, vegetation and historic and archaeological features. Total available area in a land parcel. This action is static. This means you must do it at the same location each year of this action’s duration. You must: You can use the following to get paid for this action: To test the SOM for small land parcels, you can combine them into a single sample area if the soil type and past management is similar. This means you can take so

ChatResult(chat_id=None, chat_history=[{'content': "run the worklow and output feedback for the following document: This is an action in the Sustainable Farming Incentive (SFI) scheme: expanded offer for 2024. You must read the SFI scheme information to understand the scheme rules and how to apply. 3 years £6 per hectare (ha) per year and £97 per SFI agreement per year This action’s aim is that you: You can do this action on agricultural land located below the moorland line that’s: For land above the moorland line, you can do CMOR1 to complete a survey to identify, assess and record the soil, vegetation and historic and archaeological features. Total available area in a land parcel. This action is static. This means you must do it at the same location each year of this action’s duration. You must: You can use the following to get paid for this action: To test the SOM for small land parcels, you can combine them into a single sample area if the soil type and past management is similar. 

In [7]:
from alb_checker import process, groupchat

response = process("test", groupchat)

admin (to chat_manager):

run the worklow and output feedback for the following document: This is an action in the Sustainable Farming Incentive (SFI) scheme: expanded offer for 2024. You must read the SFI scheme information to understand the scheme rules and how to apply. 3 years £6 per hectare (ha) per year and £97 per SFI agreement per year This action’s aim is that you: You can do this action on agricultural land located below the moorland line that’s: For land above the moorland line, you can do CMOR1 to complete a survey to identify, assess and record the soil, vegetation and historic and archaeological features. Total available area in a land parcel. This action is static. This means you must do it at the same location each year of this action’s duration. You must: You can use the following to get paid for this action: To test the SOM for small land parcels, you can combine them into a single sample area if the soil type and past management is similar. This means you can take so

In [9]:
response.chat_history

[{'content': "run the worklow and output feedback for the following document: This is an action in the Sustainable Farming Incentive (SFI) scheme: expanded offer for 2024. You must read the SFI scheme information to understand the scheme rules and how to apply. 3 years £6 per hectare (ha) per year and £97 per SFI agreement per year This action’s aim is that you: You can do this action on agricultural land located below the moorland line that’s: For land above the moorland line, you can do CMOR1 to complete a survey to identify, assess and record the soil, vegetation and historic and archaeological features. Total available area in a land parcel. This action is static. This means you must do it at the same location each year of this action’s duration. You must: You can use the following to get paid for this action: To test the SOM for small land parcels, you can combine them into a single sample area if the soil type and past management is similar. This means you can take soil from each

In [10]:
response.summary

'The Forestry Commission is not directly impacted by this document as it primarily pertains to the Sustainable Farming Incentive (SFI) scheme and soil management practices on agricultural land. However, there are some indirect implications that could be of interest.\n\n1. "This is an action in the Sustainable Farming Incentive (SFI) scheme: expanded offer for 2024."\n    - This section is of interest as the SFI scheme promotes sustainable farming practices, which aligns with our aim of promoting sustainable forestry practices. Understanding the actions and incentives within this scheme could provide insights for our own initiatives.\n\n2. "You can do this action on agricultural land located below the moorland line that’s: For land above the moorland line, you can do CMOR1 to complete a survey to identify, assess and record the soil, vegetation and historic and archaeological features."\n    - This section is relevant as it discusses land management practices that could potentially impa